In [54]:


# import libraries for data vizualization and analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
from scipy.integrate import solve_ivp
import plotly.graph_objects as go





In [48]:
# read trail 5 data
df1_CH1 = pd.read_csv('./trials/2-22/trial-5/F0000CH1.CSV')
df1_CH2 = pd.read_csv('./trials/2-22/trial-5/F0000CH2.CSV')


# show the first 15 rows of the data
df1_CH1.head(15) 

,Record Length,2.500000e+03,Unnamed: 2,-0.005000000000,4.70000,Unnamed: 5
0,Sample Interval,4.000000e-06,NaN,-0.004996,4.66,NaN
1,Trigger Point,1.250000000000e+03,NaN,-0.004992,4.68,NaN
2,NaN,NaN,NaN,-0.004988,4.64,NaN
3,NaN,NaN,NaN,-0.004984,4.64,NaN
4,NaN,NaN,NaN,-0.004980,4.62,NaN
5,Source,CH1,NaN,-0.004976,4.60,NaN
6,Vertical Units,V,NaN,-0.004972,4.60,NaN
7,Vertical Scale,5.000000e-01,NaN,-0.004968,4.58,NaN
8,Vertical Offset,-4.680000e+00,NaN,-0.004964,4.56,NaN
9,Horizontal Units,s,NaN,-0.004960,4.54,NaN


In [49]:
# print the types of the data
df1_CH1.dtypes
df1_CH2.dtypes

Record Length         object
2.500000e+03          object
Unnamed: 2           float64
  -0.005000000000    float64
   5.54000           float64
Unnamed: 5           float64
dtype: object

In [50]:
import plotly.graph_objects as go

# Create a figure object
fig = go.Figure()

# Customize the plot
fig.update_layout(
    title='Title',
    xaxis_title='X-axis',
    yaxis_title='Y-axis',
    template='plotly_dark',
    width=800,
)

# Plot the data
fig.add_trace(go.Scatter(
    x=df1_CH1['   4.70000'],
    y=df1_CH2['   5.54000'],
    mode='lines',
    name='sadfas',
    line=dict(color='green')
))

# Show the plot
fig.show()


In [70]:
def real_chua(t, in_vars):
    x, y, z = in_vars

    C1 = 10e-9   # 10nF
    C2 = 100e-9  # 100nF

    R = 1800     # 1.8k Ohms, POT 1 Value
    G = 1 / R

    # Chua Diode
    R1, R2, R3, R4, R5, R6 = 220, 220, 2200, 22000, 22000, 3300
    Esat = 9  # 9V batteries
    E1 = R3 / (R2 + R3) * Esat
    E2 = R6 / (R5 + R6) * Esat

    m12 = -1 / R6
    m02 = 1 / R4
    m01 = 1 / R1
    m11 = -1 / R3

    m1 = m12 + m11

    if E1 > E2:
        m0 = m11 + m02
    else:
        m0 = m12 + m01

    mm1 = m01 + m02
    Emax = max(E1, E2)
    Emin = min(E1, E2)

    if abs(x) < Emin:
        g = x * m1
    elif abs(x) < Emax:
        g = x * m0 + Emin * (m1 - m0 if x > 0 else m0 - m1)
    else:
        g = x * mm1 + Emax * (m0 - mm1 if x > 0 else mm1 - m0) + Emin * (m1 - m0 if x > 0 else m0 - m1)

    # Gyrator
    R7, R8, R9, C = 100, 1000, 1000, 100e-9  # Ohms, nF
    R10 = 1800 # pot 2 value
    
    L = R7 * R9 * C * R10 / R8  # Inductance

    # Chua's Circuit Equations
    xdot = (1 / C1) * (G * (y - x) - g)
    ydot = (1 / C2) * (G * (x - y) + z)
    zdot = -(1 / L) * y

    return [xdot, ydot, zdot]

In [82]:


# Solving the differential equations
t_span = [0, 0.05]
y0 = [-0.5, -0.2, 0]

sol = solve_ivp(real_chua, t_span, y0, t_eval=np.linspace(t_span[0], t_span[1], 2000))

# Create 3D plot using Plotly

fig = go.Figure(data=[go.Scatter3d(
    x=sol.y[0],
    y=sol.y[1],
    z=sol.y[2],
    mode='lines',
    line=dict(width=2, color=sol.y[0], colorscale='Plasma'),
)])

fig.update_layout(
    scene=dict(
        xaxis_title='v1',
        yaxis_title='v2',
        zaxis_title='iL',
        aspectratio=dict(x=1, y=1, z=1),
        camera=dict(
            eye=dict(x=1.2, y=1.2, z=1.2),
            center=dict(x=0, y=0, z=0),
        )
    ),
    title="Chua's Circuit Trajectory",
    template='seaborn',
)

fig.show()



# Plotting the variables on a 1D plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=sol.t, y=sol.y[0], name='v1'))
fig.add_trace(go.Scatter(x=sol.t, y=sol.y[1], name='v2'))
fig.add_trace(go.Scatter(x=sol.t, y=sol.y[2], name='iL'))

# Adding labels and title to the plot
fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Value',
    title="Chua's Circuit Variables",
)

# Displaying the plot
fig.show()



PlotlyKeyError: Invalid property specified for object of type plotly.graph_objs.Scatter3d: 'Time'

Did you mean "line"?

    Valid properties:
        connectgaps
            Determines whether or not gaps (i.e. {nan} or missing
            values) in the provided data arrays are connected.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets the source reference on Chart Studio Cloud for
            `customdata`.
        error_x
            :class:`plotly.graph_objects.scatter3d.ErrorX` instance
            or dict with compatible properties
        error_y
            :class:`plotly.graph_objects.scatter3d.ErrorY` instance
            or dict with compatible properties
        error_z
            :class:`plotly.graph_objects.scatter3d.ErrorZ` ins

In [ ]:
class Lorenz_Attractor(ThreeDScene):
    def construct(self):
        axes = ThreeDAxes(x_min=-3.5,x_max=3.5,y_min=-3.5,y_max=3.5,z_min=0,z_max=6,axis_config={"include_tip": True,"include_ticks":True,"stroke_width":1})
        dot = Sphere(radius=0.05,fill_color=BLUE).move_to(0*RIGHT + 0.1*UP + 0.105*OUT)
        
        self.set_camera_orientation(phi=65 * DEGREES,theta=30*DEGREES,gamma = 90*DEGREES)  
        self.begin_ambient_camera_rotation(rate=0.05)            #Start move camera

        dtime = 0.01
        numsteps = 30

        self.add(axes,dot)

        def lorenz(x, y, z, s=10, r=28, b=2.667):
            x_dot = s*(y - x)
            y_dot = r*x - y - x*z
            z_dot = x*y - b*z
            return x_dot, y_dot, z_dot

        def update_trajectory(self, dt):
            new_point = dot.get_center()
            if get_norm(new_point - self.points[-1]) > 0.01:
                self.add_smooth_curve_to(new_point)

        traj = VMobject()
        traj.start_new_path(dot.get_center())
        traj.set_stroke(BLUE, 1.5, opacity=0.8)
        traj.add_updater(update_trajectory)
        self.add(traj)

        def update_position(self,dt):
            x_dot, y_dot, z_dot = lorenz(dot.get_center()[0]*10, dot.get_center()[1]*10, dot.get_center()[2]*10)
            x = x_dot * dt/10
            y = y_dot * dt/10
            z = z_dot * dt/10
            self.shift(x/10*RIGHT + y/10*UP + z/10*OUT)

        dot.add_updater(update_position)
        self.wait(420)

NameError: name 'ThreeDScene' is not defined